# Linear Classification

In this lab you will implement parts of a linear classification model using the regularized empirical risk minimization principle. By completing this lab and analysing the code, you gain deeper understanding of these type of models, and of gradient descent.


## Problem Setting

The dataset describes diagnosing of cardiac Single Proton Emission Computed Tomography (SPECT) images. Each of the patients is classified into two categories: normal (1) and abnormal (0). The training data contains 80 SPECT images from which 22 binary features have been extracted. The goal is to predict the label for an unseen test set of 187 tomography images.

In [5]:
import urllib.request
import pandas as pd
import numpy as np
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

testfile = urllib.request.URLopener()
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.train", "SPECT.train")
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.test", "SPECT.test")

df_train = pd.read_csv('SPECT.train',header=None)
df_test = pd.read_csv('SPECT.test',header=None)

train = df_train.values
test = df_test.values

y_train = train[:,0]
X_train = train[:,1:]
y_test = test[:,0]
X_test = test[:,1:]



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
#First I'd like to understand my data and the problem at hand.
#We got a very small data set with only 87 examples.
# All features are binary, and so is our target feature, normal or abnormal tomography.
# Since we have our target feature in our data as a label, we are talking about supervised learning.
# And since we only have two classes, we are in a binary classification situation.



### Exercise 1

Analyze the function learn_reg_ERM(X,y,lambda) which for a given $n\times m$ data matrix $\textbf{X}$ and binary class label $\textbf{y}$ learns and returns a linear model $\textbf{w}$.
The binary class label has to be transformed so that its range is $\left \{-1,1 \right \}$. 
The trade-off parameter between the empirical loss and the regularizer is given by $\lambda > 0$. 
To adapt the learning rate the Barzilai-Borwein method is used.

Try to understand each step of the learning algorithm and comment each line.


In [7]:
def learn_reg_ERM(X,y,lbda):
    max_iter = 200  # Maximum number of iterations
    e  = 0.001      # Convergence threshold
    alpha = 1.      # Initial learning rate

    # Initialize random weights from normal distribution
    w = np.random.randn(X.shape[1])
    
    for k in np.arange(max_iter):
        # Compute predictions (linear combination of features and weights)
        h = np.dot(X,w)
        
        # Compute loss and gradient of loss
        l, lg = loss(h, y)
        print('loss: {}'.format(np.mean(l)))
        
        # Compute regularization term and its gradient
        r, rg = reg(w, lbda)
        
        # Compute total gradient (data term + regularization)
        g = np.dot(X.T, lg) + rg 
        
        # Barzilai-Borwein adaptive learning rate adjustment
        if (k > 0):
            alpha = alpha * (np.dot(g_old.T, g_old))/(np.dot((g_old - g).T, g_old)) 
        
        # Update weights
        w = w - alpha * g
        
        # Check for convergence (if weight change is small enough)
        if (np.linalg.norm(alpha * g) < e):
            break
            
        # Store current gradient for next iteration
        g_old = g
        
    return w

### Exercise 2

Fill in the code for the function loss(h,y) which computes the hinge loss and its gradient. 
This function takes a given vector $\textbf{y}$ with the true labels $\in \left \{-1,1\right \}$ and a vector $\textbf{h}$ with the function values of the linear model as inputs. The function returns a vector $\textbf{l}$ with the hinge loss $\max(0, 1 − y_{i} h_{i})$ and a vector $\textbf{g}$ with the gradients of the hinge loss w.r.t $\textbf{h}$. (Note: The partial derivative of the hinge loss with respect to $\textbf{h}$  is $g_{i} = −y $ if $l_{i} > 0$, else $g_{i} = 0$)

In [8]:
def loss(h, y):
    # Compute hinge loss: max(0, 1 - y*h)
    l = np.maximum(0, 1 - y * h)
    
    # Compute gradient: -y if loss > 0, else 0
    g = np.where(l > 0, -y, 0)
    
    return l, g

### Exercise 3

Fill in the code for the function reg(w,lambda) which computes the $\mathcal{L}_2$-regularizer and the gradient of the regularizer function at point $\textbf{w}$. 


$$r = \frac{\lambda}{2} \textbf{w}^{T}\textbf{w}$$

$$g = \lambda \textbf{w}$$

In [9]:
def reg(w, lbda):
    # Compute L2 regularization term: (λ/2)*w^T*w
    r = 0.5 * lbda * np.dot(w.T, w)
    
    # Compute gradient: λ*w
    g = lbda * w
    
    return r, g

### Exercise 4

Fill in the code for the function predict(w,x) which predicts the class label $y$ for a data point $\textbf{x}$ or a matrix $X$ of data points (row-wise) for a previously trained linear model $\textbf{w}$. If there is only a data point given, the function is supposed to return a scalar value. If a matrix is given a vector of predictions is supposed to be returned.

In [10]:
def predict(w, X):
    # Compute predictions using sign function
    preds = np.sign(np.dot(X, w))
    
    # For single data point, return scalar instead of array
    if preds.shape == (1,):
        return preds[0]
    return preds

### Exercise 5

#### 5.1 
Train a linear model on the training data and classify all 187 test instances afterwards using the function predict. 
Please note that the given class labels are in the range $\left \{0,1 \right \}$, however the learning algorithm expects a label in the range of $\left \{-1,1 \right \}$. Then, compute the accuracy of your trained linear model on both the training and the test data. 

In [13]:
# Convert labels from {0,1} to {-1,1}
y_train = 2 * y_train - 1
y_test = 2 * y_test - 1

# Train the model
w = learn_reg_ERM(X_train, y_train, lbda=0.01)

# Make predictions
train_preds = predict(w, X_train)
test_preds = predict(w, X_test)

# Convert predictions back to {0,1} for accuracy calculation
train_preds = (train_preds + 1) // 2
test_preds = (test_preds + 1) // 2
y_train_orig = (y_train + 1) // 2
y_test_orig = (y_test + 1) // 2

# Calculate accuracies
train_acc = np.mean(train_preds == y_train_orig)
test_acc = np.mean(test_preds == y_test_orig)

print(f"Training accuracy: {train_acc:.4f}")
print(f"Test accuracy: {test_acc:.4f}")

loss: 1.310149461001628
loss: 42.59919118729134
loss: 15.175660966844521
loss: 276.7214353695148
loss: 88.69365671241931
loss: 507.3790221979565
loss: 159.1526646856061
loss: 3278.283945303735
loss: 883.8754022964546
loss: 4280.757140659347
loss: 1018.2329979110366
loss: 5278.112337460503
loss: 933.4985967737855
loss: 3552.1231858265687
loss: 690.4722523914891
loss: 4938.374330339598
loss: 980.433493802217
loss: 4120.685422526997
loss: 891.3360742764071
loss: 4969.777422402064
loss: 902.4005490348067
loss: 3486.0497970167125
loss: 690.3124531068664
loss: 4926.377456887198
loss: 978.7409527828447
loss: 4118.794837300052
loss: 891.526885293429
loss: 4969.5421715634
loss: 902.2802761935345
loss: 3486.0514641600544
loss: 690.3618656729195
loss: 4926.367805758426
loss: 978.7167805468592
loss: 4118.801425230447
loss: 891.5374474132532
loss: 4969.541154256977
loss: 902.2757941631655
loss: 3486.0534831261693
loss: 690.3642017733997
loss: 4926.367605838301
loss: 978.7156882367146
loss: 4118.801

#### 5.2
Compare the accuracy of the linear model with the accuracy of a random forest and a decision tree on the training and test data set.

In [12]:
##################
#INSERT CODE HERE#
##################